In [5]:
# !pip install tqdm rusenttokenize nltk

     |████████████████████████████████| 1.4MB 12.1MB/s eta 0:00:01
     |████████████████████████████████| 92kB 36.4MB/s eta 0:00:01
     |████████████████████████████████| 675kB 50.6MB/s eta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import os
import json
import shutil
import tensorflow as tf
from bert.tokenization import FullTokenizer

import sys; sys.path.insert(0, "/tf/datadrive/datascientist/relation-extraction/")
from src.data.io import parse_collection, simplify, read_file_v3
from src.data.check import check_tokens, check_arcs, check_split
from src.data.preprocessing import split_example_v2, apply_bpe, enumerate_entities, fit_encodings, apply_encodings
from src.data.base import NerEncodings, NerPrefixJoiners, Languages
from src.model.base import ModeKeys
from src.model.utils import get_session
from src.model.coreference_resolution import BertForCoreferenceResolutionMentionRanking
from src.utils import train_test_valid_split, classification_report_to_string

%load_ext autoreload
%autoreload 2

W0513 07:50:57.183933 139799476565824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0513 07:50:57.186815 139799476565824 deprecation_wrapper.py:119] From /tf/datadrive/datascientist/relation-extraction/src/model/base.py:69: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
print(tf.__version__)
assert tf.test.is_gpu_available()

1.14.0


In [4]:
MODEL_DIR = "/tf/datadrive/nn_lfs/experiments/bert_for_cr_mention_ranking_10_epochs_fixed"
os.makedirs(MODEL_DIR, exist_ok=True)

### Получение выборки

* **загрузка**

In [5]:
data_dir = "/tf/datadrive/brat-v1.3_Crunchy_Frog/data/examples/rucor_v5_fixed_edges/"

examples = parse_collection(
    data_dir=data_dir,
    n=None,
    ner_encoding=NerEncodings.BIO,
    ner_prefix_joiner=NerPrefixJoiners.HYPHEN,
    tokens_pattern=None,
    allow_nested_entities=True,
    allow_nested_entities_one_label=True,
    allow_many_entities_per_span_one_label=True,
    allow_many_entities_per_span_different_labels=False,
    ignore_bad_examples=True,
    read_fn=read_file_v3
)

  2%|▏         | 4/182 [00:00<00:05, 30.28it/s]

num .txt files: 182
num .ann files: 182
num annotated texts: 182


100%|██████████| 182/182 [00:02<00:00, 75.33it/s] 

successfully parsed 182 examples from 182 files.
error counts: defaultdict(<class 'int'>, {})


In [6]:
len(examples)

182

* **check**

In [7]:
examples_filtered = []
for x in examples:
    check_tokens(x)
    try:
        check_arcs(x, one_child=True, one_parent=False)
        examples_filtered.append(x)
    except AssertionError as e:
        print(e)

[163] head T408092 <bos>Младший мой<eos> has already dep T408091 <bos>Витька<eos>, but tried to assign dep T408094 <bos>сам<eos>
[165] head T409624 <bos>Я<eos> has already dep T409622 <bos>Мальчик<eos>, but tried to assign dep T409626 <bos>мне<eos>
[169] head T409784 <bos>мальчиком<eos> has already dep T409782 <bos>его<eos>, but tried to assign dep T409786 <bos>своим мальчиком<eos>
[175] head T409508 <bos>она<eos> has already dep T409507 <bos>ей<eos>, but tried to assign dep T409511 <bos>она<eos>
[179] head T415608 <bos>Солнца<eos> has already dep T415577 <bos>Солнцу<eos>, but tried to assign dep T415610 <bos>Солнца<eos>
[181] head T410545 <bos>ее<eos> has already dep T410541 <bos>эта дама<eos>, but tried to assign dep T410549 <bos>ее<eos>


In [8]:
len(examples_filtered)

176

* **simplify**

In [9]:
examples_simplified = list(map(simplify, examples_filtered))

* **filter**

In [10]:
examples_simplified_filtered = []
for x in examples_simplified:
    is_good = True
    try:
        check_tokens(x)
    except AssertionError as e:
        print("tokens " + str(e))
        is_good = False
    try:
        check_arcs(x, one_child=True, one_parent=False)
    except AssertionError as e:
        print("arcs " + str(e))
        is_good = False
    if len(x.entities) == 0:
        is_good = False
    if is_good:
        examples_simplified_filtered.append(x)

* **assign id_chain**

In [11]:
from src.utils import get_connected_components

def assign_id_chain(examples):
    for x in examples:
        id2entity = {}
        g = {}
        for entity in x.entities:
            g[entity.id] = set()
            id2entity[entity.id] = entity
        for arc in x.arcs:
            g[arc.head].add(arc.dep)

        components = get_connected_components(g)

        for id_chain, comp in enumerate(components):
            for id_entity in comp:
                id2entity[id_entity].id_chain = id_chain

In [12]:
assign_id_chain(examples_simplified_filtered)

In [13]:
def ds_info(examples, part: str = None):
    print("part:", part)
    print("num examples:", len(examples))
    print("num entities:", sum(len(x.entities) for x in examples))
    print("num relations:", sum(len(x.arcs) for x in examples))
    print("=" * 50)

In [14]:
ds_info(examples_simplified_filtered)

part: None
num examples: 174
num entities: 14588
num relations: 11394


* **train / test / valid split**

In [15]:
examples_train, examples_valid, examples_test = train_test_valid_split(
    examples=examples_simplified_filtered,
    train_frac=0.8,
    test_frac=0.1
)

In [16]:
ds_info(examples_train, "train")
ds_info(examples_valid, "valid")
ds_info(examples_test, "test")

part: train
num examples: 139
num entities: 11778
num relations: 9203
part: valid
num examples: 18
num entities: 1343
num relations: 1053
part: test
num examples: 17
num entities: 1467
num relations: 1138


* **split by chunks**

In [17]:
fix_pointers = True

def split(examples, window: int, stride: int = 1, ignore_chunks_wo_entities: bool = True):
    chunks = []
    for x in examples:
        chunks_i = split_example_v2(
            x, 
            window=window, 
            stride=stride, 
            lang=Languages.RU, 
            tokens_expression=None,
            fix_pointers=fix_pointers
        )
        for chunk in chunks_i:
            if len(chunk.entities) > 0:
                x.chunks.append(chunk)
            else:
                if not ignore_chunks_wo_entities:
                    x.chunks.append(chunk)

In [18]:
window = 5
stride = 1
ignore_chunks_wo_entities = True

split(examples_train + examples_valid + examples_test, window, stride, ignore_chunks_wo_entities)

W0513 07:51:01.509605 139799476565824 tokenizer.py:121] Something went wrong while tokenizing
W0513 07:51:01.531503 139799476565824 tokenizer.py:121] Something went wrong while tokenizing
W0513 07:51:01.556564 139799476565824 tokenizer.py:121] Something went wrong while tokenizing
W0513 07:51:01.569420 139799476565824 tokenizer.py:121] Something went wrong while tokenizing
W0513 07:51:02.001575 139799476565824 tokenizer.py:121] Something went wrong while tokenizing
W0513 07:51:02.011615 139799476565824 tokenizer.py:121] Something went wrong while tokenizing
W0513 07:51:02.025994 139799476565824 tokenizer.py:121] Something went wrong while tokenizing
W0513 07:51:02.061110 139799476565824 tokenizer.py:121] Something went wrong while tokenizing
W0513 07:51:02.076445 139799476565824 tokenizer.py:121] Something went wrong while tokenizing
W0513 07:51:02.105899 139799476565824 tokenizer.py:121] Something went wrong while tokenizing
W0513 07:51:02.141688 139799476565824 tokenizer.py:121] Some

In [19]:
chunks = []
for x in examples_train + examples_valid + examples_test:
    chunks += x.chunks

In [20]:
for x in chunks:
    try:
        check_tokens(x)
    except AssertionError as e:
        print("tokens " + str(e))
    try:
        check_arcs(x, one_child=True, one_parent=False)
    except AssertionError as e:
        print("arcs " + str(e))
    try:
        check_split(x, window=window, fixed_sent_pointers=fix_pointers)
    except AssertionError as e:
        print("split " + str(e))

In [21]:
# for x in chunks_train + chunks_valid + chunks_test:
#     check_example(x)

In [22]:
ds_info(chunks)

part: None
num examples: 6479
num entities: 64973
num relations: 35950


* **bpe**

In [23]:
bert_dir = "/tf/datadrive/nn_lfs/pretrained/rubert_cased_L-12_H-768_A-12_v2/"
vocab_file = os.path.join(bert_dir, "vocab.txt")
shutil.copy(vocab_file, os.path.join(MODEL_DIR, "vocab.txt"))

'/tf/datadrive/nn_lfs/experiments/bert_for_cr_mention_ranking_10_epochs_fixed/vocab.txt'

In [24]:
tokenizer = FullTokenizer(vocab_file=vocab_file, do_lower_case=False)

W0513 07:51:13.111860 139799476565824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [25]:
for x in chunks:
    apply_bpe(
        x, 
        tokenizer=tokenizer, 
        ner_prefix_joiner=NerPrefixJoiners.HYPHEN,
        ner_encoding=NerEncodings.BIO
    )

In [26]:
for x in examples_train + examples_valid + examples_test:
    x.chunks = [chunk for chunk in x.chunks if all(len(t.token_ids) > 0 for t in chunk.tokens)]

In [27]:
len(chunks), sum(len(x.chunks) for x in examples_train + examples_valid + examples_test)

(6479, 6473)

* **enumerate entities**

In [28]:
for chunk in chunks:
    enumerate_entities(chunk)

### Конфигурация модели

In [29]:
bert_config = json.load(open(os.path.join(bert_dir, "bert_config.json")))
bert_config["attention_probs_dropout_prob"] = 0.5  # default 0.1
bert_config["hidden_dropout_prob"] = 0.1  # default 0.1

In [30]:
config = {
    "model": {
        "birnn": {
            "use": False,
            "params": {}
        },
        "bert": {
            "test_mode": False,
            "dropout": 0.2,
            "scope": "bert",
            "pad_token_id": tokenizer.vocab["[PAD]"],
            "cls_token_id": tokenizer.vocab["[CLS]"],
            "sep_token_id": tokenizer.vocab["[SEP]"],
            "params": bert_config
        },
        "coref": {
            "use_birnn": False,
            "rnn": {
                "num_layers": 1,
                "cell_dim": 256,
                "dropout": 0.5,
                "recurrent_dropout": 0.0
            },
            "use_attn": True,
            "attn": {
                "hidden_dim": 128,
                "dropout": 0.3,
                "activation": "relu"
            },
            "hoi": {
                "order": 1,
                "w_dropout": 0.5,
                "w_dropout_policy": 0  # 0 - one mask; 1 - different mask
            },
            "biaffine": {
                "num_mlp_layers": 1,
                "activation": "relu",
                "head_dim": 128,
                "dep_dim": 128,
                "dropout": 0.33,
                "num_labels": 1,
                "use_dep_prior": False
            }
        }
    },
    "optimizer": {
        "init_lr": 2e-5,
        "warmup_proportion": 0.1
    },
    "training": {
        "num_epochs": 10,
        "batch_size": 8,  # уменьшен, т.к. maxlen=256
        "max_epochs_wo_improvement": 20,
        "num_train_samples": sum(len(x.chunks) for x in examples_train),
        "maxlen": 256
    },
    "valid": {
        "path_true": "/tmp/gold.conll",
        "path_pred": "/tmp/pred.conll",
        "scorer_path": "/tf/datadrive/reference-coreference-scorers/scorer.pl",
        "window": window
    },
    "inference": {
        "max_tokens_per_batch": 10000,
        "window": window,
        "maxlen": 256
    }
}

In [35]:
tf.reset_default_graph()
sess = get_session()
model = BertForCoreferenceResolutionMentionRanking(sess=sess, config=config)
model.build(mode=ModeKeys.TRAIN)
model.save_config(model_dir=MODEL_DIR)
model.reset_weights(bert_dir=bert_dir)

In [32]:
def tvars_info():
    num_bert_tvars = 0
    num_non_bert_tvars = 0
    for v in tf.trainable_variables():
        n = 1
        for d in v.shape:
            n *= d
        if v.name.startswith("model/bert"):
            num_bert_tvars += n
        else:
            num_non_bert_tvars += n
    print("num bert tvars:", num_bert_tvars)
    print("num non bert tvars:", num_non_bert_tvars)

tvars_info()

num bert tvars: 177853440
num non bert tvars: 707458


In [34]:
# import numpy as np
# from tensorflow.python.pywrap_tensorflow import NewCheckpointReader
# from tqdm import tqdm

# def check_initialization():
#     checkpoint_path = os.path.join(bert_dir, "bert_model.ckpt")
#     reader = NewCheckpointReader(checkpoint_path)

#     for v in tqdm(tf.trainable_variables()):
#         name = model._actual_name_to_checkpoint_name(v.name)
#         if name.startswith("bert"):
#             ckpt_value = reader.get_tensor(name)
#             init_value = v.eval(session=sess)
#             assert np.allclose(ckpt_value, init_value), name

# check_initialization()

### Обучение

In [33]:
def verbose_fn(d):
    print("loss:", d["loss"])
    print("score:", d["score"])
    print("num_entities:", d["num_entities"])
    print("num_chains_true:", d["num_chains_true"])
    print("num_chains_pred:", d["num_chains_pred"])
    print(classification_report_to_string(d["metrics"]))

In [36]:
model.train(
    examples_train=examples_train,
    examples_valid=examples_valid,
    model_dir=MODEL_DIR,
    verbose=True,
    verbose_fn=verbose_fn
)

train started.
model dir /tf/datadrive/nn_lfs/experiments/bert_for_cr_mention_ranking_10_epochs_fixed exists
checkpoint path: /tf/datadrive/nn_lfs/experiments/bert_for_cr_mention_ranking_10_epochs_fixed/model.ckpt


  0%|          | 0/634 [00:00<?, ?it/s]

number of ignored examples: 76
following examples are ignored due to their length is > 256 pieces:
105_6-11 => 261
178_64-69 => 281
178_65-70 => 289
178_66-71 => 299
178_67-72 => 307
1_9-14 => 267
1_10-15 => 302
1_11-16 => 384
1_12-17 => 291
95_14-19 => 283
95_15-20 => 303
95_16-21 => 296
95_24-29 => 300
95_25-30 => 269
95_26-31 => 263
95_27-32 => 273
76_69-74 => 292
76_70-75 => 281
76_71-76 => 353
76_72-77 => 324
76_73-78 => 295
118_22-27 => 305
118_23-28 => 313
118_24-29 => 315
118_25-30 => 351
118_26-31 => 264
118_125-130 => 272
118_126-131 => 341
118_127-132 => 332
118_128-133 => 329
118_129-134 => 342
97_18-23 => 307
97_19-24 => 303
97_20-25 => 347
97_21-26 => 304
97_22-27 => 263
97_31-36 => 266
97_32-37 => 264
60_20-25 => 314
60_21-26 => 339
6_27-32 => 300
6_28-33 => 400
6_29-34 => 348
6_30-35 => 327
6_32-37 => 262
106_8-13 => 290
106_9-14 => 302
106_10-15 => 354
106_11-16 => 339
106_12-17 => 387
106_13-18 => 290
106_14-19 => 302
106_15-20 => 275
106_16-21 => 283
106_18-23 => 257

100%|██████████| 634/634 [02:48<00:00,  3.76it/s]


epoch 0 finished. mean train loss: 1.2191115617752075. evaluation starts.
evaluate started.
evaluate finished. Time elapsed: 0:00:15.150949
loss: 0.5928544139289474
score: 0.5962000000000001
num_entities: 1343
num_chains_true: 290
num_chains_pred: 563
       f1         precision  recall     

bcub   0.6389     0.8197     0.5234     
blanc  0.6838     0.7797     0.6498     
ceafe  0.47       0.356      0.6912     
ceafm  0.5547     0.5547     0.5547     
muc    0.7212     0.8474     0.6277
current score: 0.5962000000000001
!!! new best score: 0.5962000000000001


...

epoch 8 finished. mean train loss: 0.3420267105102539. evaluation starts.
evaluate started.
evaluate finished. Time elapsed: 0:00:10.942943
loss: 0.6106913581222116
score: 0.709025
num_entities: 1343
num_chains_true: 290
num_chains_pred: 423
       f1         precision  recall     

bcub   0.7178     0.8221     0.637      
blanc  0.7334     0.8117     0.6964     
ceafe  0.6247     0.5265     0.7679     
ceafm  0.6634     0.6634     0.6634     
muc    0.8302     0.8902     0.7777
current score: 0.709025
!!! new best score: 0.709025


  0%|          | 0/634 [00:00<?, ?it/s]

saved new head to /tf/datadrive/nn_lfs/experiments/bert_for_cr_mention_ranking_10_epochs_fixed/model.ckpt


100%|██████████| 634/634 [02:40<00:00,  3.96it/s]


epoch 9 finished. mean train loss: 0.3170735836029053. evaluation starts.
evaluate started.
evaluate finished. Time elapsed: 0:00:10.936375
loss: 0.6164409543292534
score: 0.70235
num_entities: 1343
num_chains_true: 290
num_chains_pred: 435
       f1         precision  recall     

bcub   0.706      0.8337     0.6123     
blanc  0.7154     0.8218     0.6751     
ceafe  0.6227     0.5189     0.7784     
ceafm  0.6567     0.6567     0.6567     
muc    0.824      0.8898     0.7673
current score: 0.70235
best score: 0.709025
steps wo improvement: 1
restoring model from /tf/datadrive/nn_lfs/experiments/bert_for_cr_mention_ranking_10_epochs_fixed/model.ckpt
train finished. Time elapsed: 0:28:51.959571


In [37]:
d_test = model.evaluate(examples=examples_test)
verbose_fn(d_test)

evaluate started.
evaluate finished. Time elapsed: 0:00:11.998621
loss: 0.6534284791141826
score: 0.65395
num_entities: 1467
num_chains_true: 329
num_chains_pred: 566
       f1         precision  recall     

bcub   0.6748     0.8216     0.5725     
blanc  0.6977     0.7889     0.6559     
ceafe  0.5583     0.4414     0.7594     
ceafm  0.6128     0.6128     0.6128     
muc    0.7699     0.8712     0.6898


### predict

In [38]:
# tf.reset_default_graph()
# sess = get_session()
# model = BertForCoreferenceResolutionMentionRanking.load(sess=sess, model_dir=MODEL_DIR)

In [39]:
import copy
examples_test_copy = copy.deepcopy(examples_test)

In [40]:
for x in examples_test_copy:
    x.arcs = []
    for entity in x.entities:
        entity.id_chain = None

In [41]:
model.predict(examples=examples_test_copy, flat_chains=True)

predict started.
predict finished. Time elapsed: 0:00:07.626766


### save

In [42]:
from src.data.io import to_brat_v2

output_dir = "/tf/datadrive/brat-v1.3_Crunchy_Frog/data/examples/rucor_v5_preds_80_10_10_mention_ranking_10_epochs_flat_chains/"
to_brat_v2(examples=examples_test_copy, output_dir=output_dir)